In [1]:
# 1. Dataset, DataLoader
# 2. Model
# 3. Loss
# 4. Optimizer

In [2]:
# 1.a Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor())

testing_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor())

training_data.data.shape, testing_data.data.shape

(torch.Size([60000, 28, 28]), torch.Size([10000, 28, 28]))

In [3]:
# 1.b DataLoader
from torch.utils.data import DataLoader

training_dataloader = DataLoader(training_data, 64, shuffle=True)
testing_dataloader = DataLoader(testing_data, 64, shuffle=False)

X, y = next(iter(training_dataloader))
X.shape, y.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [4]:
# 2. Model
from torch import nn

# This is simple model that take input -> linear -> relu -> linear -> output.
class LinearModel(nn.Module):
    def __init__(self):
        super(LinearModel, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, 100),
            nn.ReLU(),
            nn.Linear(100, 10),
            nn.ReLU())
    def forward(self, x):
        out = self.model(x)
        return out
model = LinearModel()

y_hat = model(X)

In [5]:
# 3. Loss
loss_fn = nn.CrossEntropyLoss()

loss = loss_fn(y_hat, y)

loss.backward()

In [6]:
# 4. Optimizer SGD
from torch.optim import SGD

lr = 0.001
optimizer = SGD(model.parameters(), lr)

In [7]:
# 5. Combine things together:
# train model
epochs = 10
size = len(training_dataloader.dataset)
for epoch in range(epochs):
    for batch, (X, y) in enumerate(training_dataloader):
        y_hat = model(X)
        loss = loss_fn(y_hat, y)

        # backward
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 300 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

loss: 2.308181  [    0/60000]
loss: 2.229337  [19200/60000]
loss: 2.205912  [38400/60000]
loss: 2.195541  [57600/60000]
loss: 2.190738  [    0/60000]
loss: 2.018688  [19200/60000]
loss: 1.915945  [38400/60000]
loss: 1.930671  [57600/60000]
loss: 1.896908  [    0/60000]
loss: 1.883524  [19200/60000]
loss: 1.794088  [38400/60000]
loss: 1.577187  [57600/60000]
loss: 1.677914  [    0/60000]
loss: 1.632509  [19200/60000]
loss: 1.543742  [38400/60000]
loss: 1.473006  [57600/60000]
loss: 1.550810  [    0/60000]
loss: 1.537076  [19200/60000]
loss: 1.510127  [38400/60000]
loss: 1.406873  [57600/60000]
loss: 1.561509  [    0/60000]
loss: 1.575962  [19200/60000]
loss: 1.450021  [38400/60000]
loss: 1.369262  [57600/60000]
loss: 1.469433  [    0/60000]
loss: 1.482930  [19200/60000]
loss: 1.458911  [38400/60000]
loss: 1.233172  [57600/60000]
loss: 1.375724  [    0/60000]
loss: 1.175986  [19200/60000]
loss: 1.366483  [38400/60000]
loss: 1.140404  [57600/60000]
loss: 1.473560  [    0/60000]
loss: 1.23

In [23]:
# Test model
y_hat.shape, y.shape
total = 0
total += sum(y_hat.argmax(1) == y).item()
total

14

In [26]:
epochs = 10
size = len(testing_dataloader.dataset)
total = 0
for X, y in testing_dataloader:
    y_hat = model(X)
    total += sum(y_hat.argmax(1) == y).item()
print(f'Accuracy: {total/size:>2f}')

Accuracy: 0.552500
